In [1]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import math

In [2]:
data = np.load('/raided/datasets/MNIST_train_40000.npz')

In [3]:
labels = OneHotEncoder(sparse=False).fit_transform(data['train_labels'].reshape(-1, 1))
images = data['train_images']
images = np.expand_dims(images, axis=-1)

In [4]:
def split_data(data, labels, train_percent):
    batch_size = math.floor(data.shape[0] * train_percent)
    data, labels = get_minibatches(data, labels, batch_size)
    return data[0], labels[0], data[1], labels[1]

In [5]:
def get_minibatches(data, labels, batch_size):
    shuffled_indices = np.random.permutation(data.shape[0])
    split_indices = np.arange(batch_size, data.shape[0], batch_size)
    return np.split(data[shuffled_indices], split_indices), np.split(labels[shuffled_indices], split_indices)

In [6]:
train_images, train_labels, test_images, test_labels = split_data(images, labels, 0.8)

In [7]:
batch_size = 128
width = images.shape[1]
height = images.shape[2]
channels = 1

In [8]:
x = tf.placeholder(tf.float32, (None, width, height, channels))
y = tf.placeholder(tf.float32, (None, 10))

with tf.variable_scope('layer1'):
    conv1 = tf.contrib.layers.convolution2d(x, 32, 3)
    pool1 = tf.nn.max_pool(conv1, (1, 2, 2, 1), (1, 2, 2, 1), 'SAME')

with tf.variable_scope('layer2'):
    conv2 = tf.contrib.layers.convolution2d(pool1, 64, 3)
    pool2 = tf.nn.max_pool(conv2, (1, 2, 2, 1), (1, 2, 2, 1), 'SAME')

In [9]:
flat = tf.reshape(pool2, (-1, np.prod(pool2.get_shape()[1:]).value))
hidden = tf.contrib.layers.fully_connected(flat, 1024, tf.nn.relu, )

In [10]:
output = tf.contrib.layers.fully_connected(hidden, 10, tf.nn.relu)
prediction = tf.nn.softmax(output)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, y))

In [11]:
train = tf.train.AdamOptimizer().minimize(loss)

In [12]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [13]:
for i in range(100):
    x_batches, y_batches = get_minibatches(train_images, train_labels, batch_size)
    
    for x_batch, y_batch in zip(x_batches, y_batches):
        sess.run([train], feed_dict={x: x_batch, y: y_batch})
    
    pred, ce = sess.run([prediction, loss], feed_dict={x: test_images, y: test_labels})
    accuracy = accuracy_score(np.argmax(test_labels, axis=1), np.argmax(pred, axis=1))
    print('Epoch: %d\tAccuracy: %.3f\tCross entropy: %.3f' % (i, accuracy, ce))

Epoch: 0	Accuracy: 0.787	Cross entropy: 0.525
Epoch: 1	Accuracy: 0.796	Cross entropy: 0.490
Epoch: 2	Accuracy: 0.798	Cross entropy: 0.485
Epoch: 3	Accuracy: 0.796	Cross entropy: 0.486
Epoch: 4	Accuracy: 0.794	Cross entropy: 0.491
Epoch: 5	Accuracy: 0.799	Cross entropy: 0.481
Epoch: 6	Accuracy: 0.799	Cross entropy: 0.477
Epoch: 7	Accuracy: 0.799	Cross entropy: 0.481
Epoch: 8	Accuracy: 0.797	Cross entropy: 0.487
Epoch: 9	Accuracy: 0.799	Cross entropy: 0.484
Epoch: 10	Accuracy: 0.799	Cross entropy: 0.494
Epoch: 11	Accuracy: 0.800	Cross entropy: 0.481
Epoch: 12	Accuracy: 0.798	Cross entropy: 0.487
Epoch: 13	Accuracy: 0.800	Cross entropy: 0.474
Epoch: 14	Accuracy: 0.798	Cross entropy: 0.484
Epoch: 15	Accuracy: 0.800	Cross entropy: 0.478
Epoch: 16	Accuracy: 0.800	Cross entropy: 0.480
Epoch: 17	Accuracy: 0.795	Cross entropy: 0.500
Epoch: 18	Accuracy: 0.797	Cross entropy: 0.481
Epoch: 19	Accuracy: 0.799	Cross entropy: 0.479
Epoch: 20	Accuracy: 0.801	Cross entropy: 0.482
Epoch: 21	Accuracy: 0.8

In [14]:
np.argmax(sess.run(prediction, feed_dict={x: test_images[:10]}), axis=1)

array([0, 7, 9, 2, 0, 4, 7, 7, 3, 7])

In [15]:
np.argmax(test_labels[:10], axis=1)

array([6, 7, 9, 2, 6, 4, 7, 7, 3, 7])